In [1]:
import psycopg2
import duckdb
import pandas as pd
from dotenv import load_dotenv
import os


load_dotenv()

# PostgreSQL-Verbindungsparameter
pg_host = os.getenv('DB_HOST')
pg_dbname = os.getenv('DB_NAME')
pg_user = os.getenv('DB_USER')
pg_password = os.getenv('DB_PASSWORD')

# Verbindung zu PostgreSQL herstellen
pg_conn = psycopg2.connect(
    host=pg_host,
    dbname=pg_dbname,
    user=pg_user,
    password=pg_password
)

# DuckDB-Verbindung (oder neue DuckDB-Datenbank erstellen)
duckdb_conn = duckdb.connect('narrAItive_duckDB.duckdb')

# Funktion zum Laden einer Tabelle von PostgreSQL nach DuckDB
def load_table_to_duckdb(table_name):
    # Daten von PostgreSQL in einen Pandas DataFrame laden
    query = f"SELECT * FROM {table_name};"
    df = pd.read_sql_query(query, pg_conn)
    
    # Daten in DuckDB schreiben
    duckdb_conn.execute(f"CREATE TABLE {table_name} AS SELECT * FROM df")

# Tabellen, die migriert werden sollen
tables = ["audios", "images", "story", "story_segments", "voices"]

# Tabellen nacheinander migrieren
for table in tables:
    load_table_to_duckdb(table)

# Verbindungen schließen
pg_conn.close()
duckdb_conn.close()

print("Datenbankmigration abgeschlossen!")

Datenbankmigration abgeschlossen!


/var/folders/29/9sbs0n5j2dx09wpxhjw1tgbm9575g4/T/ipykernel_21930/557359602.py:31: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, pg_conn)
/var/folders/29/9sbs0n5j2dx09wpxhjw1tgbm9575g4/T/ipykernel_21930/557359602.py:31: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, pg_conn)
/var/folders/29/9sbs0n5j2dx09wpxhjw1tgbm9575g4/T/ipykernel_21930/557359602.py:31: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, pg_conn)
/var/folders/29/9sbs0n5j2dx0

In [2]:
# get tables from duckdb
conn = duckdb.connect('narrAItive_duckDB.duckdb')
cursor = conn.cursor()
cursor.execute("SHOW TABLES;")
print(cursor.fetchall())

[('audios',), ('images',), ('story',), ('story_segments',), ('voices',)]


In [3]:
# get data from duckdb
cursor.execute("SELECT * FROM story;")
print(cursor.fetchall())

[('61888c17-0d9a-4f7d-8c0d-76c03e4df02b', 10, 'Die kleine blonde Mädchen und der grüne Drache', '3-5', 'gpt-3.5-turbo-16k', datetime.datetime(2023, 11, 9, 9, 51, 21, 347722)), ('5d3f8f1f-ffba-49ce-8e11-c3b7baac55e6', 10, 'Die Ninja Turtles entdecken den Planeten Mars', '3-5', 'gpt-3.5-turbo-16k', datetime.datetime(2023, 11, 9, 9, 59, 47, 775695)), ('fe18f6a8-04e7-4cba-bb5e-939a2ac1f67d', 10, 'Die Feuerwehrhelden', '3-5', 'gpt-3.5-turbo-16k', datetime.datetime(2023, 11, 9, 11, 32, 13, 850684)), ('1ab9babc-3374-474b-8f0d-27aa1cf7967c', 10, 'Spongebob und Dagobert Duck treffen sich in der Kneipe', '6-8', 'gpt-3.5-turbo-16k', datetime.datetime(2023, 11, 9, 16, 15, 37, 128807)), ('afee59ee-45f8-4bd5-9340-63fb8cf0cfaf', 10, 'Ein Bär im Dschungel', '1-2', 'gpt-3.5-turbo-16k', datetime.datetime(2023, 11, 10, 15, 48, 37, 982716)), ('37f83491-9108-4271-9d38-f60c3d545bb6', 10, 'Peppa Wutz und Dinosaurier spielen Verstecken auf einer Party und essen Brötchen', '3-5', 'gpt-3.5-turbo-16k', datetime.

In [7]:
# create dataframe from table stories
df = conn.table("images").to_df()
df

,image_id,segment_id,url,creation_timestamp,creation_model,resolution
0,4b029826-d229-4cfb-9437-930dcc0c5d2c,10657725-fe7e-4eeb-9de6-3f2b608ec4e0,https://narraitive.s3.amazonaws.com/61888c17-0...,2023-11-09 09:51:42.207358,dall_e_3_gpt,1024x1024
1,2c2cbfe7-635e-4d5e-a376-80b26b5f357c,03114ad9-94f6-4d88-8050-e6d550548140,https://narraitive.s3.amazonaws.com/61888c17-0...,2023-11-09 09:52:05.647147,dall_e_3_gpt,1024x1024
2,86f936e0-7bbe-4ae7-8e3c-115d0c41b2e7,c66ec0fd-706a-4382-9242-ac5a704a27f3,https://narraitive.s3.amazonaws.com/61888c17-0...,2023-11-09 09:52:32.325643,dall_e_3_gpt,1024x1024
3,ad6186d9-7097-46a3-98d1-fa5a700198e8,dca8a314-a347-4f21-bd92-4741c90bb215,https://narraitive.s3.amazonaws.com/61888c17-0...,2023-11-09 09:52:54.002847,dall_e_3_gpt,1024x1024
4,f920a4a2-4e92-4a0b-accf-78e348fd04d8,c4d5e6b1-0ac4-469d-8e1e-355b9787938c,https://narraitive.s3.amazonaws.com/61888c17-0...,2023-11-09 09:53:16.192214,dall_e_3_gpt,1024x1024
...,...,...,...,...,...,...
305,72e73eb0-1093-4370-b214-faec4477451b,b1a09529-8c3f-49c1-98da-3efe30f3e4b9,https://narraitive.s3.amazonaws.com/a65b149d-9...,2023-12-06 18:10:12.831909,dall_e_3_gpt,1024x1024
306,216af273-a9da-4bcc-b6b1-e2b9de3c7a23,8dcae62e-f98f-4093-a161-c7e1fc912cbb,https://narraitive.s3.amazonaws.com/a65b149d-9...,2023-12-06 18:10:51.103463,dall_e_3_gpt,1024x1024
307,2448ce23-184f-4af1-87e2-5ffb08b0a937,ea239b80-f5c6-414e-bbcc-cbbe8c0606f9,https://narraitive.s3.amazonaws.com/a65b149d-9...,2023-12-06 18:11:28.642090,dall_e_3_gpt,1024x1024
308,311b63a2-e058-4c47-8852-60757845b749,2632ff66-c6b6-4edf-bcab-7a2c17826183,https://narraitive.s3.amazonaws.com/a65b149d-9...,2023-12-06 18:11:58.158703,dall_e_3_gpt,1024x1024
